In [1]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
from zipfile import ZipFile

In [10]:
from PIL import Image
import shutil

In [19]:
!pip3 install jupyter-tabnine
!jupyter nbextension install --py jupyter_tabnine
!jupyter nbextension enable --py jupyter_tabnine
!jupyter serverextension enable --py jupyter_tabnine

Installing C:\Users\Admin\anaconda3\lib\site-packages\jupyter_tabnine\static -> jupyter_tabnine
Up to date: C:\ProgramData\jupyter\nbextensions\jupyter_tabnine\main.css
Up to date: C:\ProgramData\jupyter\nbextensions\jupyter_tabnine\main.js
Up to date: C:\ProgramData\jupyter\nbextensions\jupyter_tabnine\README.md
Up to date: C:\ProgramData\jupyter\nbextensions\jupyter_tabnine\tabnine.yaml
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable jupyter_tabnine --py
    
Enabling notebook extension jupyter_tabnine/main...
      - Validating: ok
Enabling: jupyter_tabnine
- Writing config: C:\Users\Admin\.jupyter
    - Validating...
      jupyter_tabnine  ok


In [2]:
data_directory_path = "Data"
data_zipfile_name = "archive.zip"
data_zipfile_path = os.path.join(data_directory_path, data_zipfile_name)

In [7]:
# with ZipFile(data_zipfile_path, "r") as zip_f:
#     zip_f.extractall("Data/")

In [3]:
target_data_dirs = ["cats", "dogs"]
parent_directory = os.path.join(data_directory_path, "training_set")
parent_directory = os.path.join(parent_directory, "training_set")
print(parent_directory)

Data\training_set\training_set


In [7]:
BAD_DATA_DIR = "Bad_data"
os.makedirs(BAD_DATA_DIR, exist_ok = True)

In [12]:
# Finding full path to the images and verifying the Images  

for dirs in os.listdir(parent_directory):
    full_path_data_dirs = os.path.join(parent_directory, dirs)
    for img in os.listdir(full_path_data_dirs):
        full_path_image = os.path.join(full_path_data_dirs, img)
        try:
            image = Image.open(full_path_image)
            image.verify()
#             print(f"{full_path_image} --> is varified")
            
        except Exception as e:
#             print(f"{full_path_image} --> is BAD")
            bad_data_path = os.path.join(BAD_DATA_DIR, img)
            shutil.move(full_path_image, bad_data_path)
            

In [14]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [23]:
#  img_dataset_from_directory will take the parent directory path
#  it will split the data set, resize the images 


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    parent_directory,
    validation_split = 0.2,
    subset = "training",
    seed = 13,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    parent_directory,
    validation_split = 0.2,
    subset = "validation",
    seed = 13,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
)


Found 8005 files belonging to 2 classes.
Using 6404 files for training.
Found 8005 files belonging to 2 classes.
Using 1601 files for validation.
